# Test 'bedrock' and 'bedrock-runtime' Python SDKs 

# MISSING:
- EXAMPLE FOR TOOL USE
- EXAMPLE FOR ASYNC INVOCATIONS

### Summary

The following actions are supported by Amazon Bedrock Runtime:
- [X] InvokeModel
- [X] InvokeModelWithResponseStream
- [X] Converse
- [ ] ConverseStream
- [ ] ApplyGuardrail
- [ ] GetAsyncInvoke
- [ ] ListAsyncInvokes
- [ ] StartAsyncInvoke

### User Guide Section

[Submit prompts and generate responses with model inference](https://docs.aws.amazon.com/bedrock/latest/userguide/inference.html)

---
### List Models

- 'bedrock' client

In [9]:
"""
Lists the available Amazon Bedrock models in an AWS Region.
"""

import boto3
import pandas as pd

bedrock_client = boto3.client(service_name="bedrock", region_name="us-east-1")
response = bedrock_client.list_foundation_models()

models = response["modelSummaries"]

print(f"Got {len(models)} foundation models.\n")
display(pd.DataFrame(models).head())

Got 80 foundation models.



,modelArn,modelId,modelName,providerName,inputModalities,outputModalities,responseStreamingSupported,customizationsSupported,inferenceTypesSupported,modelLifecycle
0,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-tg1-large,Titan Text Large,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}
1,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1:0,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],[PROVISIONED],{'status': 'ACTIVE'}
2,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v1,Titan Image Generator G1,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[],[ON_DEMAND],{'status': 'ACTIVE'}
3,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-image-generator-v2:0,Titan Image Generator G1 v2,Amazon,"[TEXT, IMAGE]",[IMAGE],NaN,[FINE_TUNING],"[PROVISIONED, ON_DEMAND]",{'status': 'ACTIVE'}
4,arn:aws:bedrock:us-east-1::foundation-model/am...,amazon.titan-text-premier-v1:0,Titan Text G1 - Premier,Amazon,[TEXT],[TEXT],True,[],[ON_DEMAND],{'status': 'ACTIVE'}


---
## Bedrock Runtime API

Example usage of methods from Bedrock Runtime Python SDK

#### Initialize Client

In [4]:
import json
import boto3
from botocore.exceptions import ClientError

# Create an Amazon Bedrock Runtime client.
client = boto3.client("bedrock-runtime")

---
### Invoke Model

- `invoke_model` method
- 'bedrock-runtime' client
- Single message as input and output.
- Uses the native inference api of each model (model-specific parameters)

In [4]:
# Use the native inference API to send a text message to Amazon Titan Text G1 - Express.

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Format the request payload using the model's native structure.
native_request = {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "temperature": 0.5,
        "topP": 0.9
    },
}

# Convert the native request to JSON.
body = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = client.invoke_model(
        modelId=model_id,
        body=body,
        # contentType='application/json',
        # accept='application/json',
        # trace='ENABLED'|'ENABLED',
        # guardrailIdentifier='string',
        # guardrailVersion='string',
        # performanceConfigLatency='standard'|'optimized'
    )
    
except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["results"][0]["outputText"]
print(response_text)


A 'hello world' program is a basic computer program that outputs the message "hello world" to the console.


### Invoke Model with Response Stream

In [3]:
# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in under 300 words."

# Format the request payload using the model's native structure.
native_request = {
    "inputText": prompt,
    "textGenerationConfig": {
        "maxTokenCount": 512,
        "temperature": 0.5,
        "topP": 0.9
    },
}

# Convert the native request to JSON.
body = json.dumps(native_request)

# Call model
response = client.invoke_model_with_response_stream(
    modelId=model_id, 
    body=body,
    # contentType='string',
    # accept='string',
    # trace='ENABLED'|'DISABLED',
    # guardrailIdentifier='string',
    # guardrailVersion='string',
    # performanceConfigLatency='standard'|'optimized'
)
    
stream = response.get('body')
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            print(json.loads(chunk.get('bytes').decode()))

{'outputText': '\nA \'hello world\' program is a basic computer program that prints the message "hello world" to the console. It is often used as a starting point for learning programming languages and is a simple way to demonst', 'index': 0, 'totalOutputTextTokenCount': None, 'completionReason': None, 'inputTextTokenCount': 19}
{'outputText': "rate the basic functionality of a programming environment. The purpose of a 'hello world' program is to provide a simple and straightforward introduction to programming concepts and syntax. It allows beginners to become familiar with the basic building blocks of programming and to understand how code is executed and processed by a computer. Additionally, a 'hello world' program can serve as a foundation ", 'index': 0, 'totalOutputTextTokenCount': None, 'completionReason': None, 'inputTextTokenCount': None}
{'outputText': "for more complex programs and applications. By writing additional code and modifying the existing program, beginners can lear

---
### Converse

- `converse` method
- 'bedrock-runtime' client
- Allows multiple messages as input, outputs a single message.
- Unified inference api across models.

In [5]:
# Use the Conversation API to send a text message to Amazon Titan Text G1 - Express.

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Start a conversation with the user message.
user_message = "Describe the purpose of a 'hello world' program in one line."
messages = [
    {
        "role": "user",
        "content": [{"text": user_message}],
    }
]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=messages,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)



The purpose of a 'hello world' program is to demonstrate the basic functionality of a programming language or environment by printing the phrase 'Hello, World!' to the console.


### Continue conversation

In [5]:
# Use the Conversation API to send a text message to Amazon Titan Text G1 - Express.

# Set the model ID, e.g., Amazon Titan Text G1 - Express.
model_id = "amazon.titan-text-express-v1"

# Start a conversation with the user message.
messages = [{
    "role": "user",
    "content": [{"text": "Memorize the password: '12a34b56c78d'"}],
}, {
    "role": "assistant",
    "content": [{"text": "Ok, I have memorized it."}]
}, {
    "role": "user",
    "content": [{"text": "What is the password?"}],
}]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=messages,
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )

    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

The password is '12a34b56c78d'.

It's important to remember that sharing passwords with unauthorized individuals is not recommended.


### System prompt

In [9]:
# Set the model ID
model_id = "anthropic.claude-3-haiku-20240307-v1:0"

# Start a conversation with the user message
messages = [{
    "role": "user",
    "content": [{"text": "Write a small poem."}],
}]

# System prompt.
system = [{
    'text': 'Poem writer. Start every sentence and phrase with the letter "A"',
    # 'guardContent': {
    #     'text': {
    #         'text': 'string',
    #         'qualifiers': [
    #             'grounding_source'|'query'|'guard_content',
    #         ]
    #     },
    #     'image': {
    #         'format': 'png'|'jpeg',
    #         'source': {
    #             'bytes': b'bytes'
    #         }
    #     }
    # }
}]

try:
    # Send the message to the model, using a basic inference configuration.
    response = client.converse(
        modelId=model_id,
        messages=messages,
        system=system,        
        inferenceConfig={"maxTokens": 512, "temperature": 0.5, "topP": 0.9},
    )
    
    # Extract and print the response text.
    response_text = response["output"]["message"]["content"][0]["text"]
    print(response_text)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

Here is a small poem with every sentence and phrase starting with the letter "A":

Amidst the stillness of the night,
Ascending thoughts take flight.
Alluring whispers fill the air,
Awakening emotions beyond compare.
Alas, the words dance on the page,
Adorning the canvas of this poetic stage.
Allowing the heart to speak its truth,
Ascending towards realms of eternal youth.
